# Google Scholar
https://scholar.google.com/
Google Scholar provides a simple way to broadly search for scholarly literature. From one place, you can search across many disciplines and sources: articles, theses, books, abstracts and court opinions, from academic publishers, professional societies, online repositories, universities and other web sites. Google Scholar helps you find relevant work across the world of scholarly research

# Langchain

### Agents
https://python.langchain.com/v0.1/docs/modules/agents/
https://python.langchain.com/v0.1/docs/modules/agents/agent_types/

### Semantic Scholar
https://python.langchain.com/v0.2/docs/integrations/tools/semanticscholar/

### Google Genai
https://python.langchain.com/v0.1/docs/integrations/chat/google_generative_ai/

# Vertexai
https://cloud.google.com/vertex-ai/docs/general/custom-service-account

# GeminiAI
https://ai.google.dev/gemini-api/docs/api-key

```Wrapper for Google Search API.

Adapted from: Instructions adapted from https://stackoverflow.com/questions/
37083058/
programmatically-searching-google-in-python-using-custom-search

TODO: DOCS for using it
1. Install google-api-python-client
- If you don't already have a Google account, sign up.
- If you have never created a Google APIs Console project,
read the Managing Projects page and create a project in the Google API Console.
- Install the library using pip install google-api-python-client

2. Enable the Custom Search API
- Navigate to the APIs & Services→Dashboard panel in Cloud Console.
- Click Enable APIs and Services.
- Search for Custom Search API and click on it.
- Click Enable.
URL for it: https://console.cloud.google.com/apis/library/customsearch.googleapis
.com

3. To create an API key:
- Navigate to the APIs & Services → Credentials panel in Cloud Console.
- Select Create credentials, then select API key from the drop-down menu.
- The API key created dialog box displays your newly created key.
- You now have an API_KEY

Alternatively, you can just generate an API key here:
https://developers.google.com/custom-search/docs/paid_element#api_key

4. Setup Custom Search Engine so you can search the entire web
- Create a custom search engine here: https://programmablesearchengine.google.com/.
- In `What to search` to search, pick the `Search the entire Web` option.
After search engine is created, you can click on it and find `Search engine ID`
  on the Overview page.
Init docstring:
Create a new model by parsing and validatin```
g input data from keyword arguments.

In [1]:
%pip install --upgrade --quiet  semanticscholar langchainhub google-cloud-aiplatform langchain-google-genai langchain langchain-community langchain-experimental langchain-openai langchain-pinecone langchain-text-splitters -q --user

Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import dotenv_values

In [2]:
config = dotenv_values(".env")

In [75]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from google.oauth2 import service_account
import vertexai
import json
import os
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_community.tools.google_search import GoogleSearchResults
from langchain.retrievers.web_research import WebResearchRetriever
from langchain_chroma import Chroma

from langchain_community.tools.semanticscholar.tool import SemanticScholarQueryRun
from langchain_community.utilities.semanticscholar import SemanticScholarAPIWrapper
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import RetrievalQAWithSourcesChain
import chromadb.utils.embedding_functions as embedding_functions

In [5]:
openai_api_key = config["KEY_OPENAI"]
os.environ["GEMINI_API_KEY"] = config['GOOGLE_API_KEY']
os.environ["GOOGLE_API_KEY"] = config.get("GOOGLE_API_KEY")
os.environ["GOOGLE_CSE_ID"] = config.get("GOOGLE_CSE_ID")

In [6]:
with open("keys/complete-tube-421007-9a7c35cd44e2.json") as source:
    info = json.load(source)

vertex_credentials = service_account.Credentials.from_service_account_info(info)
vertexai.init(
    project=config["PROJECT"],
    location=config["REGION"],
    credentials=vertex_credentials,
)
google_api_key = config["GEMINI-API-KEY"]
os.environ["GEMINI_API_KEY"] = google_api_key

In [27]:
llm = ChatGoogleGenerativeAI(
                    model="gemini-1.5-flash-001", credentials=vertex_credentials, google_api_key=config['GOOGLE_API_KEY'], temperature=0

                )

In [71]:
api_wrapper = SemanticScholarAPIWrapper(doc_content_chars_max=10000, top_k_results=10)


### https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/

In [36]:
#agent = prompt| llm | tools

In [37]:
# Open AI
#agent = create_openai_functions_agent(llm, tools, prompt)
# generic
# agent = create_tool_calling_agent(llm, tools, prompt)

In [98]:
template = """ Question: {query} 
                {context}
                    with the context build a response with the following requirements:
                    - Show me a list of papers and techniques. 
                    - Based on your findings write a description, main symptoms and how to treat them
                    - Create 3 different sections: List of papers, Description/ symptoms and treatment. 
                  
"""
prompt_query = ChatPromptTemplate.from_template(template)

In [141]:
retrieval_chain = (
    
        {"context": SemanticScholarQueryRun(api_wrapper=api_wrapper), "query": RunnablePassthrough()}
        | prompt_query
        | llm
    | StrOutputParser()
    
)

In [142]:
out = retrieval_chain.invoke("What is the SAPHO syndrome and what are the symptoms?")

In [143]:
print(out)

## SAPHO Syndrome: A Rare Inflammatory Condition

**SAPHO** stands for **Synovitis, Acne, Pustulosis, Hyperostosis, and Osteitis**. It's a rare inflammatory condition that primarily affects the bones, skin, and joints. While the exact cause is unknown, it's believed to be an autoimmune disorder where the body's immune system mistakenly attacks healthy tissues. 

**List of Papers:**

* **"SAPHO syndrome: a review"** by  A.  M.  G.  van  der  Heijden  et  al. (2004) - This paper provides a comprehensive overview of SAPHO syndrome, including its clinical presentation, diagnosis, and treatment.
* **"SAPHO syndrome: a clinical and radiological study"** by  J.  C.  van  der  Heijden  et  al. (2002) - This paper focuses on the clinical and radiological features of SAPHO syndrome, highlighting the importance of imaging studies in diagnosis.
* **"Treatment of SAPHO syndrome: a review"** by  M.  A.  van  der  Heijden  et  al. (2006) - This paper reviews the various treatment options for SAPHO sy

In [145]:
# Search
search = GoogleSearchAPIWrapper()

In [146]:
# Vectorstore
vectorstore = Chroma(
    embedding_function=GoogleGenerativeAIEmbeddings(
                        model="models/embedding-001",
                        credentials=vertex_credentials,
                        google_api_key=google_api_key,
                    ), persist_directory="./chroma_db_google"
)

In [159]:
# Initialize
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore, llm=llm, search=search
)

In [160]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)


In [161]:
template2 = """ Question: {question} 
                based on your findings return an anwers and citate the sources

                  
"""
prompt_query = ChatPromptTemplate.from_template(template2)

In [162]:
user_input = "What is the Síndrome SAPHO and what are the symptoms? Based on your findings write a description, main symptoms and how to treat them "
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, retriever=web_research_retriever
)
#result = qa_chain.invoke({"question": user_input})

In [153]:
result.keys()

dict_keys(['question', 'answer', 'sources'])

In [152]:
result['answer']

"SAPHO syndrome is a rare inflammatory disorder that affects the bones, skin, and joints. It is characterized by a combination of symptoms, including:\n\n* **Synovitis:** Inflammation of the joints, often affecting the sterno-clavicular joint (where the collarbone meets the breastbone), sacroiliac joint (where the spine meets the pelvis), and other joints.\n* **Acne:** Severe, persistent acne, often with pustules (pus-filled bumps).\n* **Pustulosis:** Pustules on the skin, often on the palms of the hands and soles of the feet.\n* **Hyperostosis:** Thickening of the bones, often in the skull, spine, and pelvis.\n* **Osteitis:** Inflammation of the bone, often affecting the sterno-clavicular joint, sacroiliac joint, and other joints.\n\nThe exact cause of SAPHO syndrome is unknown, but it is thought to be an autoimmune disorder, meaning the body's immune system attacks its own tissues. It may also be triggered by infections, such as bacteria or viruses.\n\nTreatment for SAPHO syndrome ty

In [154]:
result['sources']

'**'

In [163]:
map_chain = RunnableParallel(scholar=retrieval_chain, google=qa_chain)

In [164]:
response = map_chain.invoke({"question": user_input, "query" :"What is the SAPHO syndrome and what are the symptoms?"})

Fetching pages: 100%|####################################################################| 2/2 [00:01<00:00,  1.83it/s]


In [165]:
response.keys()

dict_keys(['scholar', 'google'])

In [166]:
response['google'].get("answer")

"SAPHO syndrome is a rare inflammatory disorder that affects the bones, skin, and joints. It is characterized by a combination of symptoms, including:\n\n* **Synovitis:** Inflammation of the joints, often affecting the sterno-clavicular joint (where the collarbone meets the breastbone), sacroiliac joint (where the spine meets the pelvis), and other joints.\n* **Acne:** Severe, persistent acne, often with pustules (pus-filled bumps).\n* **Pustulosis:** Pustules on the skin, often on the palms of the hands and soles of the feet.\n* **Hyperostosis:** Thickening of the bones, often in the skull, spine, and pelvis.\n* **Osteitis:** Inflammation of the bone, often affecting the sterno-clavicular joint, sacroiliac joint, and other joints.\n\nThe exact cause of SAPHO syndrome is unknown, but it is thought to be an autoimmune disorder, meaning the body's immune system attacks its own tissues. It may also be triggered by infections, such as bacteria or viruses.\n\nTreatment for SAPHO syndrome ty

In [167]:
response['scholar']

'## SAPHO Syndrome: A Rare Inflammatory Condition\n\n**List of Papers:**\n\n* **"SAPHO syndrome: a review"** by  A.  M.  G.  van  der  Heijden  et  al.  (2004) - This comprehensive review provides an overview of the syndrome, its clinical features, and diagnostic criteria.\n* **"SAPHO syndrome: a clinical and radiological study"** by  J.  C.  van  der  Heijden  et  al.  (2002) - This study focuses on the clinical and radiological manifestations of SAPHO syndrome.\n* **"Treatment of SAPHO syndrome: a review"** by  M.  A.  van  der  Heijden  et  al.  (2006) - This review discusses various treatment options for SAPHO syndrome, including medications and surgical interventions.\n\n**Description and Symptoms:**\n\nSAPHO syndrome, an acronym for **S**ynovitis, **A**cne, **P**ustulosis, **H**yperostosis, and **O**steitis, is a rare inflammatory condition characterized by a combination of bone and skin lesions. It is a complex disorder with a wide range of symptoms and severity. \n\n**Main Symp

In [107]:
from operator import itemgetter

In [168]:
template_last="""
you are a researcher wich take diferent inputs and combine the to provide an anwers
instruction to build the final answer to the question{question}
from {context}
- extract 2 dictionaries with keys scholar and google 
- content of scholar is the input 1
- from dictionary google extract the key answer and this is the input 2
- with the input 1 and input2  build a final answer in human languange 
"""
prompt_last = ChatPromptTemplate.from_template(template_last)
second_chain = prompt_last | llm

In [169]:
complete_chain = (
    {   "question" :itemgetter("question"),
         "query":  itemgetter("query"),  
     "context" : map_chain
         
    } | RunnablePassthrough.assign(d=second_chain)
)

In [170]:
response = complete_chain.invoke({"question": user_input, "query" :"What is the SAPHO syndrome and what are the symptoms?"})

In [171]:
response.keys()

dict_keys(['question', 'query', 'context', 'd'])

In [172]:
print(response['d'].content)

## SAPHO Syndrome: A Rare Inflammatory Condition

SAPHO syndrome, an acronym for **S**ynovitis, **A**cne, **P**ustulosis, **H**yperostosis, and **O**steitis, is a rare inflammatory condition characterized by a combination of bone and skin lesions. It is a complex disorder with a wide range of symptoms and severity. 

**Main Symptoms:**

* **Skin lesions:** Acne, pustulosis, and other skin eruptions are common.
* **Bone lesions:**  Inflammation and bone overgrowth (hyperostosis) can occur in various parts of the skeleton, particularly the spine, ribs, sternum, and skull.
* **Joint pain and swelling:**  Synovitis, inflammation of the joint lining, can cause pain and swelling in the affected joints.
* **Fever:**  Fever is a common symptom, especially during active phases of the disease.
* **Fatigue:**  Chronic fatigue is often reported by individuals with SAPHO syndrome.

**Treatment:**

Treatment for SAPHO syndrome is tailored to the individual patient and the severity of their symptoms.

In [173]:
response['context']

{'scholar': '## SAPHO Syndrome: A Rare Inflammatory Condition\n\n**List of Papers:**\n\n* **"SAPHO syndrome: a review"** by  A.  M.  G.  van  der  Heijden,  et  al.  (2004) - This comprehensive review provides an overview of the syndrome, its clinical features, and diagnostic criteria.\n* **"SAPHO syndrome: a clinical and radiological study"** by  J.  C.  L.  van  der  Heijden,  et  al.  (2002) - This study focuses on the clinical and radiological manifestations of SAPHO syndrome.\n* **"Treatment of SAPHO syndrome: a review"** by  M.  A.  van  der  Heijden,  et  al.  (2006) - This review discusses various treatment options for SAPHO syndrome, including medications and surgical interventions.\n\n**Description and Symptoms:**\n\nSAPHO syndrome, an acronym for **S**ynovitis, **A**cne, **P**ustulosis, **H**yperostosis, and **O**steitis, is a rare inflammatory condition characterized by a combination of bone and skin lesions. It is a complex disorder with a wide range of symptoms and severi